# Descriptive information and statistics
This notebook will correspond to the [Descriptive Statistics](http://www.ats.ucla.edu/stat/sas/modules/descript.htm) page. That tutorial covers basic features of SAS and applies them to the cars dataset. The dataset contains variables on **make, price, miles per gallon, repair rating (in 1978)**, and whether the car was **foreign** or **domestic**.

### SAS Code

    DATA auto ;
      input MAKE $ PRICE MPG REP78 FOREIGN ;
    DATALINES;
    [data]
    RUN;
    
    PROC PRINT DATA=auto(obs=10);
    RUN;
    
#### Python Code

In [1]:
from io import StringIO

import pandas as pd

In [3]:
datalines = StringIO('''
AMC    4099 22 3 0
AMC    4749 17 3 0
AMC    3799 22 3 0
Audi   9690 17 5 1
Audi   6295 23 3 1
BMW    9735 25 4 1
Buick  4816 20 3 0
Buick  7827 15 4 0
Buick  5788 18 3 0
Buick  4453 26 3 0
Buick  5189 20 3 0
Buick 10372 16 3 0
Buick  4082 19 3 0
Cad.  11385 14 3 0
Cad.  14500 14 2 0
Cad.  15906 21 3 0
Chev.  3299 29 3 0
Chev.  5705 16 4 0
Chev.  4504 22 3 0
Chev.  5104 22 2 0
Chev.  3667 24 2 0
Chev.  3955 19 3 0
Datsun 6229 23 4 1
Datsun 4589 35 5 1
Datsun 5079 24 4 1
Datsun 8129 21 4 1
''')

auto = pd.read_table(datalines,
                     delim_whitespace=True,
                     names=['make', 'price', 'mpg', 'rep78', 'foreign'])

datalines.close()

auto.head(10)

,make,price,mpg,rep78,foreign
0,AMC,4099,22,3,0
1,AMC,4749,17,3,0
2,AMC,3799,22,3,0
3,Audi,9690,17,5,1
4,Audi,6295,23,3,1
5,BMW,9735,25,4,1
6,Buick,4816,20,3,0
7,Buick,7827,15,4,0
8,Buick,5788,18,3,0
9,Buick,4453,26,3,0


## Using Proc Freq for Frequencies
### SAS Code
    PROC FREQ DATA=auto;
      TABLES make ;
    RUN;

    PROC FREQ DATA=auto;
      TABLES rep78 ;
    RUN;

    PROC FREQ DATA=auto;
      TABLES foreign ;
    RUN;

#### Python Code
`PROQ FREQ` returns four different pieces of information about a variable: Frequency, Percent, Cumulative Frequency, and Cumulative Percent. Python can do so as well, but only if you ask it. Below we will individually replicate each of these measures, and then write our own custom function to return all of them.

In [6]:
# Frequency, sorted alphabetically
auto['make'].value_counts(sort=False)

AMC       3
BMW       1
Audi      2
Datsun    4
Buick     7
Cad.      3
Chev.     6
Name: make, dtype: int64

In [7]:
# Percent, sorted alphabetically
auto['make'].value_counts(sort=False, normalize=True)

AMC       0.115385
BMW       0.038462
Audi      0.076923
Datsun    0.153846
Buick     0.269231
Cad.      0.115385
Chev.     0.230769
Name: make, dtype: float64

In [8]:
# Cumulative Frequency, sorted alphabetically
auto['make'].value_counts(sort=False).cumsum()

AMC        3
BMW        4
Audi       6
Datsun    10
Buick     17
Cad.      20
Chev.     26
Name: make, dtype: int64

In [9]:
# Cumulative Percent, sorted alphabetically
auto['make'].value_counts(sort=False, normalize=True).cumsum()

AMC       0.115385
BMW       0.153846
Audi      0.230769
Datsun    0.384615
Buick     0.653846
Cad.      0.769231
Chev.     1.000000
Name: make, dtype: float64

<div class="pynote">
<b>Python Note</b>:
One thing you may notice in the cumulative functions is the fact that we're adding multiple methods or functions on the previous piece. The first part, `auto['make']` is a Series, calling the `.value_counts()` method on that Series actually returns another Series that contains the value counts. Then, we're calling `.cumsum()` on the `value_counts()` Series, returning the cumulative sum of the value counts. A method like `cumsum()` works on any Series, regardless of whether you have aggregated the data in any way.
<br><br>
You can find the type of any object by calling the `type()` function on it. For example:
<code><pre>> type(auto['make'].value_counts())
pandas.core.series.Series</code></pre>
</div>

## Detour: Creating our own Python Functions
One of the advantages of using a general programming language is that we can create our own objects that meet our needs. You may find this similar to using the macro language in SAS, however, Python is much more flexible since it is a general programming language

Below we'll create our own function for returning an object like the SAS version of `PROC FREQ`. The contents of the function itself isn't anything new, but saving the results of a `value_counts()` to an object as well as the structure of a function may be new.



<div class="pynote">
<b>Python Note</b>:
In general, functions in python follow the following format:
<pre><code>def add(a, b):
    return a + b</pre></code>
    
A function starts with a `def`, followed by the function name and parenthesis, and ends with a `return` statement describing what the function should return. On the first line, the items in parenthesis are the arguments you want your function to take: typically objects you'll pass in and options for how you want to interact with those objects.

<br><br>
Our function below will take in a `Series`, or the column from the data frame we want to analyze. We'll also add in an option to sort our results called `sort_option`, and we'll pass that function argument to our calls of `value_counts()` within our function. We'll also set the default to not sort our results in decending frequency but by standard alpanumeric sort.</div>

In [45]:
def proc_freq(series, sort_option=False):    
    # Create the four series of value counts with different arguments
    frequency = series.value_counts(sort=sort_option)
    percent = series.value_counts(sort=sort_option, normalize=True)
    cumulative_frequency = series.value_counts(sort=sort_option).cumsum()
    cumulative_percent = series.value_counts(sort=sort_option, normalize=True).cumsum()
    
    # Create a list of the series and concatenate them all row-wise
    frequency_data = [frequency, percent, cumulative_frequency, cumulative_percent]
    freq_table = pd.concat(frequency_data, axis=1)
    
    # Rename columns in the resulting data frame & rename index
    freq_table.columns = ['Frequency', 'Percent', 'Cumulative Frequency', 'Cumulative Percent']
    freq_table.index.name = series.name
    
    return freq_table

<div class="pynote">
<b>Python Note</b>:
This last step may seem like a bit of a leap -- where did all of that code come from? I don't want to give the illusion that the entire function popped into my head all at once. It took some experimenting beforehand to understand how it would all fit together and I worked through a few small tweaks before I finalized the version you see here.
<br><br>
You should try and approach learning a new language with an open mind and curious attitude -- experimenting with code is an essential part of learning. There really is no 'messing up', you can redefine objects and functions as much as you need and you can iterate as long as it takes to get a satisfactory result.
</div>

Now that we've defined our function, let's see if it works!

In [51]:
proc_freq(auto['make'])

,Frequency,Percent,Cumulative Frequency,Cumulative Percent
make,,,,
AMC,3,0.115385,3,0.115385
BMW,1,0.038462,4,0.153846
Audi,2,0.076923,6,0.230769
Datsun,4,0.153846,10,0.384615
Buick,7,0.269231,17,0.653846
Cad.,3,0.115385,20,0.769231
Chev.,6,0.230769,26,1.000000


We should also validate that our function works as expected with the `sort_option` set to `False`.

In [52]:
proc_freq(auto['make'], sort_option=True)

,Frequency,Percent,Cumulative Frequency,Cumulative Percent
make,,,,
Buick,7,0.269231,7,0.269231
Chev.,6,0.230769,13,0.500000
Datsun,4,0.153846,17,0.653846
Cad.,3,0.115385,20,0.769231
AMC,3,0.115385,23,0.884615
Audi,2,0.076923,25,0.961538
BMW,1,0.038462,26,1.000000


<div class="challenge">
<b>Challenge</b>:
Can you adapt our `proc_freq` function to handle missing values? Hint: `value_counts` has an option that may be relevant.
</div>

## Crosstabs
### SAS Code
    PROC FREQ DATA=auto;
      TABLES rep78*foreign ;
    RUN; 
#### Python Code

In [55]:
pd.crosstab(auto['rep78'], auto['foreign'])

foreign,0,1
rep78,,
2,3,0
3,14,1
4,2,4
5,0,2


While SAS typically errs on the side of giving you too much, python usually doesn't give you anything you don't ask for. Using the `TABLES` statement in SAS returns the frequency, overall percent, row percent, and column percent by cell, as well as a total columns. All of these options are available in python, but you ask to ask for them to be included.

For example, if we wanted a row and column for totals, we can pass `True` to the `margins` argument.

In [61]:
pd.crosstab(auto['rep78'], auto['foreign'], margins=True)

foreign,0,1,All
rep78,,,
2,3,0,3
3,14,1,15
4,2,4,6
5,0,2,2
All,19,7,26


We can also calculate row or column percentages by passing `'index'`, `'column'`, or `'all'` to the `normalize` argument.

In [62]:
pd.crosstab(auto['rep78'], auto['foreign'], normalize='index')

foreign,0,1
rep78,,
2,1.000000,0.000000
3,0.933333,0.066667
4,0.333333,0.666667
5,0.000000,1.000000


## Using PROC MEANS for summary statistics
### SAS Code
     PROC MEANS DATA=auto;
      VAR mpg;
    RUN;
    
#### Python Code

In [35]:
auto['mpg'].describe()

count    26.000000
mean     20.923077
std       4.757504
min      14.000000
25%      17.250000
50%      21.000000
75%      23.000000
max      35.000000
Name: mpg, dtype: float64

## Summary Statistics by Group (CLASS statement)

If we wanted to do something similar to the `CLASS` option in `PROC MEANS`, we can take advantage of the `groupby` method on our DataFrame.

`groupby` is a very powerful method for data frames that allows an analyst to do complex manipuluation by groups far beyond what we're doing here.

### SAS Code
    PROC MEANS DATA=auto;
      CLASS foreign ;
      VAR mpg;
    RUN;

#### Python Code

In [66]:
auto_group_foreign = auto.groupby('foreign')
auto_group_foreign['mpg'].describe()

foreign       
0        count    19.000000
         mean     19.789474
         std       4.035660
         min      14.000000
         25%      16.500000
         50%      20.000000
         75%      22.000000
         max      29.000000
1        count     7.000000
         mean     24.000000
         std       5.507571
         min      17.000000
         25%      22.000000
         50%      23.000000
         75%      24.500000
         max      35.000000
Name: mpg, dtype: float64

<div class="resources">
<a href="http://pandas.pydata.org/pandas-docs/stable/groupby.html">This</a> article from the <code>pandas</code> documentation does a good job of describing the philosophy, inner workings, and possibilities using <code>groupby</code>.
</div>


## Using PROC UNIVARIATE for detailed summary statistics
In the *Introduction to the Features of SAS* notebook we replicated some of the functionality of `PROC UNIVARIATE` piece by piece. Instead of replicating the functionality again, we'll create a function that will tie together the critical pieces of this procedure and allow us to get a unified output similar to `PROC UNIVARIATE`. 

We'll try to replicate:
- Summary Statistics
- Skew / Kurtosis
- Quantiles
- Extreme observations (top / bottom 5)


### SAS Code
    PROC UNIVARIATE DATA=auto;
      VAR mpg;
    RUN; 
    
#### Python Code

In [114]:
def proc_univariate(series):
    # creating the descriptive statistics Series
    # we're going to suppress the 25/75 percentiles
    #   by passing in an empty list to the percentiles argument
    descriptives = series.describe(percentiles=[])
    
    # add skew and kurtosis to our description series
    descriptives['kurtosis'] = series.kurtosis()
    descriptives['skewness'] = series.skew()
    
    quantiles_standard = series.quantile([1, .75, .5, .25, 0])
    quantiles_extremes = series.quantile([.99, .95, .90, .10, .05, .01])
    
    lowest_obs = series.sort_values().head()
    highest_obs = series.sort_values(ascending=False).head()

    print("Univariate Statistics")
    print("Variable:", series.name)
    print("\n--- Moments ---")
    print(descriptives)
    print("\n--- Quantiles ---")
    print(quantiles_standard)
    print()
    print(quantiles_extremes)
    print("\n--- Extreme Observations ---")
    print("Lowest\n(Index, Value)")
    print(lowest_obs)
    print("\nHighest\n(Index, Value)")
    print(highest_obs)

In [115]:
proc_univariate(auto['mpg'])

Univariate Statistics
Variable: mpg

--- Moments ---
count       26.000000
mean        20.923077
std          4.757504
min         14.000000
50%         21.000000
max         35.000000
kurtosis     1.792700
skewness     0.935473
Name: mpg, dtype: float64

--- Quantiles ---
1.00    35.00
0.75    23.00
0.50    21.00
0.25    17.25
0.00    14.00
Name: mpg, dtype: float64

0.99    33.50
0.95    28.25
0.90    25.50
0.10    15.50
0.05    14.25
0.01    14.00
Name: mpg, dtype: float64

--- Extreme Observations ---
Lowest
(Index, Value)
14    14
13    14
7     15
17    16
11    16
Name: mpg, dtype: int64

Highest
(Index, Value)
23    35
16    29
9     26
5     25
24    24
Name: mpg, dtype: int64


<div class="challenge">
<b>Challenge</b>:
Can you adapt our `proc_univariate` function to include functionality similar to <code>PROC UNIVARIATE</code>'s <code>CLASS</code> statement? That is, get it to return this output for a grouped subset of observations?
</div>

That completes our tutorial. At this point you should be able to replicate the general functinoality of the [Descriptive Statistics](http://www.ats.ucla.edu/stat/sas/modules/descript.htm) tutorial from UCLA.

------

In [113]:
# This cell imports the styling for this notebook. You can safely ignore it.

from IPython.display import HTML

def css_styling():
    styles = open("../../_styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

In [112]:
!python -V

Python 3.5.2 :: Anaconda 4.1.1 (x86_64)
